In [1]:
import warnings
warnings.filterwarnings("ignore")

import read_data as imp
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.covariance import LedoitWolf, OAS
import pyfolio

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def find_median(x):
    aux = [0 for k in range(x.shape[0])]
    for i in range(x.shape[0]):
        for j in range(x.shape[0]):
            aux[i] += np.linalg.norm(x.iloc[i,]-x.iloc[j,])
    return x.iloc[aux == min(aux),].iloc[0,]

In [3]:
base = 252

symbols = pd.read_csv('Nemos.csv')['Nemo'].tolist()
years = [x for x in range(2013,2019)]
hist_data = imp.organizarTodo(symbols,years)

Daily_Assets = pd.DataFrame()
for asset,df in hist_data.items():
    Daily_Assets[asset] = df['Close']
    
Daily_Log_Assets = Daily_Assets.pct_change()

In [4]:
Initial_Date = pd.to_datetime('2014-01-01')

Rebalancing_dates = [Initial_Date]
curr_month = Initial_Date.month

the_start = np.where(Initial_Date == Daily_Assets.index)[0].tolist()[0]

for i in range((the_start+1), len(Daily_Assets.index)):
    if Daily_Assets.index[i].month != curr_month:
        Rebalancing_dates.append(Daily_Assets.index[i-1])
        curr_month = Daily_Assets.index[i].month

com = 0.0015
BidAskSpread = 0.00016

In [ ]:
Markowitz_Returns = pd.DataFrame(index = Daily_Assets.index[Daily_Assets.index >= Initial_Date],
                                 columns = ['Clasico', 'Mediana', 'Mediana Indicadora', 'Indicadora Invertida'],
                                 dtype = 'float')

Curr_Wi = np.zeros(len(Daily_Assets.columns))
RCurr_Wi = np.zeros(len(Daily_Assets.columns))
R2Curr_Wi = np.zeros(len(Daily_Assets.columns))
R3Curr_Wi = np.zeros(len(Daily_Assets.columns))

WC = pd.DataFrame(index=Daily_Assets.index[Daily_Assets.index >= Initial_Date], columns= Daily_Assets.columns)
WR = pd.DataFrame(index=Daily_Assets.index[Daily_Assets.index >= Initial_Date], columns= Daily_Assets.columns)
WR2 = pd.DataFrame(index=Daily_Assets.index[Daily_Assets.index >= Initial_Date], columns= Daily_Assets.columns)
WR3 = pd.DataFrame(index=Daily_Assets.index[Daily_Assets.index >= Initial_Date], columns= Daily_Assets.columns)

Memory_time_window = base

for i in Daily_Assets.index[Daily_Assets.index >= Initial_Date]:
            
    if i in Rebalancing_dates:
        
        end = np.where(i == Daily_Assets.index)[0].tolist()[0]
        ini = end - Memory_time_window
        
        dfaux = Daily_Assets.iloc[ini:end,:].pct_change().dropna(how="all").copy()
               
        
        # Classic
        
        
       
        OAS_Cov = OAS(assume_centered=False).fit(dfaux).covariance_
        
        mu = base * dfaux.mean()
        S = base * OAS_Cov

        ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
        ef.max_sharpe()
        Curr_Wi = np.array([x for y,x in ef.clean_weights().items()])
        
        
        # Margin Median
        
        mu = base * dfaux.median()
        OAS_Cov = OAS(assume_centered=False).fit(dfaux).covariance_
        
        S = base * OAS_Cov

        ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
        ef.max_sharpe()
        RCurr_Wi = np.array([x for y,x in ef.clean_weights().items()])
        
        # Indicator Median
        mu = base * find_median(dfaux)
        OAS_Cov = OAS(assume_centered=False).fit(dfaux).covariance_
        
        S = base * OAS_Cov

        ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
        ef.max_sharpe()
        R2Curr_Wi = np.array([x for y,x in ef.clean_weights().items()])
        
        R3Curr_Wi = -R2Curr_Wi.copy()
        
        # Saving Weights
        
        WC.loc[i,] = Curr_Wi
        WR.loc[i,] = RCurr_Wi
        WR2.loc[i,] = R2Curr_Wi
        WR3.loc[i,] = R3Curr_Wi
        
        Markowitz_Returns.loc[i, ['Clasico']] = np.dot(Daily_Log_Assets.loc[i,],Curr_Wi)*(1-com)*(1-BidAskSpread)
        Markowitz_Returns.loc[i, ['Mediana']] = np.dot(Daily_Log_Assets.loc[i,],RCurr_Wi)*(1-com)*(1-BidAskSpread)
        Markowitz_Returns.loc[i, ['Mediana Indicadora']] = np.dot(Daily_Log_Assets.loc[i,],R2Curr_Wi)*(1-com)*(1-BidAskSpread)
        Markowitz_Returns.loc[i, ['Indicadora Invertida']] = np.dot(Daily_Log_Assets.loc[i,],R3Curr_Wi)*(1-com)*(1-BidAskSpread)
    else:
        Markowitz_Returns.loc[i, ['Clasico']] = np.dot(Daily_Log_Assets.loc[i,],Curr_Wi)
        Markowitz_Returns.loc[i, ['Mediana']] = np.dot(Daily_Log_Assets.loc[i,],RCurr_Wi)
        Markowitz_Returns.loc[i, ['Mediana Indicadora']] = np.dot(Daily_Log_Assets.loc[i,],R2Curr_Wi)
        Markowitz_Returns.loc[i, ['Indicadora Invertida']] = np.dot(Daily_Log_Assets.loc[i,],R3Curr_Wi)

## Weights Behivor

In [ ]:
# for asset in symbols:
#     plt.figure(figsize=(17,17))
#     plt.plot(WC[asset][100:150])
#     plt.plot(WR[asset][100:150])
#     plt.plot(WR2[asset][100:150])
#     plt.legend(['Classic', 'Robust', 'Robust2'])
#     plt.show()

## Comparison

In [ ]:
Markowitz_Returns.to_excel('Resultados portafolios Long Short LDL.xlsx')
WC.to_excel('Historico Pesos Clasico Long Short LDL.xlsx')
WR.to_excel('Historico Pesos Mediana Long Short LDL.xlsx')
WR2.to_excel('Historico Pesos Indicadora Long Short LDL.xlsx')
WR3.to_excel('Historico Pesos Indicadora Inversa Long Short LDL.xlsx') 

In [ ]:
pyfolio.create_full_tear_sheet(Markowitz_Returns['Mediana'], benchmark_rets=Markowitz_Returns['Clasico'])

In [ ]:
pyfolio.create_full_tear_sheet(Markowitz_Returns['Mediana Indicadora'], benchmark_rets=Markowitz_Returns['Clasico'])

In [ ]:
pyfolio.create_full_tear_sheet(Markowitz_Returns['Indicadora Invertida'], benchmark_rets=Markowitz_Returns['Clasico'])

In [ ]:
pyfolio.create_full_tear_sheet(Markowitz_Returns['Clasico'])